In [10]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from function import lineal, sigmoide
import os 

In [9]:

#Capa de entrada
x=np.zeros(100, dtype=int)
#Inicializamos los pesos (100 por cada neurona, tenemos 5 neuronas)
w = np.random.rand(500)
w11 = w[:100]
w12 = w[100:200]
w13 = w[200:300]
w14 = w[300:400]
w15 = w[400:500]
W1 = [w11,w12,w13,w14,w15]

w = np.random.rand(25)
w21 = w[:5]
w22 = w[5:10]
w23 = w[10:15]
w24 = w[15:20]
w25 = w[20:25]
W2 = [w21,w22,w23,w24,w25]
#Bias
b = random.random()
#Codificacion de clases
#b = 00000
#d = 00001
#f = 00010
Y=[[0,0,0,0,0],[0,0,0,0,1],[0,0,0,1,0]]

In [4]:

#Este tendria que ir si ejecutamos en archivo .py
#os.path.join(os.path.dirname(__file__),"src","data","distorsionadas","100")
#Input de cantidad de datos a usar
size = 100
letter =['b','d','f']
#b: es la clase 0 (cero)
#d: es la clase 1 (uno)
#f: es la clase 2 (dos)
clase = [0,1,2]

#ruta = os.path.join(os.path.abspath(''),"data","distorsionadas",str(size),'letras'+'.csv')
#data = pd.read_csv(ruta,sep=';', header=None)




In [10]:
b = (pd.read_csv(os.path.join(os.path.abspath(''),"data","distorsionadas",'100','letras_b.csv'),sep=';',header=None)).to_numpy()
d = (pd.read_csv(os.path.join(os.path.abspath(''),"data","distorsionadas",'100','letras_d.csv'),sep=';',header=None)).to_numpy()
f = (pd.read_csv(os.path.join(os.path.abspath(''),"data","distorsionadas",'100','letras_f.csv'),sep=';',header=None)).to_numpy()
letter =[b,d,f]
data =[]
aux=[]
i=0
while i<99:
    for j in range(3):
     data.append((letter[j])[i])
    i=i+3
data.append(b[70])
#Se define como dateframe para poder leer el csv mas facil
#data = pd.DataFrame(data)
#data.to_csv('data.csv',sep=';',header=None,index=False)

In [6]:
#Convertimos el dataframe en array numpy
data_train = data[:int(len(data)*0.8)]
data_test = data[int(len(data)*0.8)+1:int(len(data)*0.8)+int(len(data)*0.15)]
data_validation = data[int(len(data)*0.8)+int(len(data)*0.15)+1:99]

In [7]:
"""Tendria que crear un vector de clases "Y" y multiplicarlo por el vector data, haciendo
corresponder cada fila con su clase.
Primero voy multiplicar los pesos con las entradas para ver que valores me da"""

'Tendria que crear un vector de clases "Y" y multiplicarlo por el vector data, haciendo\ncorresponder cada fila con su clase.\nPrimero voy multiplicar los pesos con las entradas para ver que valores me da'

In [12]:

#for i in range(len(data_train)):
primera_capa = []
for j in range(len(W1)):
    resultado = (np.multiply(data_train[0],W1[j]))
    resultado = np.sum(resultado, dtype=np.float32)
    resultado = resultado+b
    primera_capa.append(lineal(resultado))

segunda_capa = []
#for x in range(len(primera_capa)):
for j in range(len(W2)):
    resultado = (np.multiply(primera_capa,W2[j]))
    resultado = np.sum(resultado, dtype=np.float32)
    resultado = resultado+b
    segunda_capa.append(sigmoide(resultado))


segunda_capa


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()